<a href="https://colab.research.google.com/github/EADhrubo/modelwithKNN/blob/master/modelwithKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [0]:
#library
import numpy as np
#import librosa 
import scipy.io.wavfile as sciwave
from scipy.fftpack import dct
import matplotlib.pyplot as plt
from numpy.linalg import norm

In [0]:
# wave signal reading
sample_signal = {}
sample_rate = {}
S = 5 # number of speech
N = 15 # amount of speech
K = 1
for i in range(1,(S+1)):
  for j in range(1,(N+1)):
    [sample_rate[str(K)] , sample_signal[str(K)]] = sciwave.read(str(i) + "Recording" + " " + "(" + str(j) + ")" + ".wav")
    K += 1


In [0]:
# autocorrelation step
freqs = {}
auto1 = {}
autocorr_signal = {}

for i in range(1,K):
  freqs[str(i)] = np.fft.rfft(sample_signal[str(i)])
  auto1[str(i)] = freqs[str(i)] * np.conj(freqs[str(i)])
  autocorr_signal[str(i)] = np.fft.irfft(auto1[str(i)])
  

In [0]:
# starting point and ending point detection
j = 0
for i in range(1,K):
  while j<8000:
    d = np.abs(autocorr_signal[str(i)][j,1] - autocorr_signal[str(i)][(j+1),1])
    if  d > 5:
      if j < 500:
        y = autocorr_signal[str(i)]
      elif 500 < j < 1000:
        y = autocorr_signal[str(i)][(j-500):]
      elif 1000< j < 10000:
        y = autocorr_signal[str(i)][(j-1000):]
      elif 10000< j < 20000:
        y = autocorr_signal[str(i)][(j-10000):]
      elif 20000 < j < 50000:
        y = autocorr_signal[str(i)][(j-20000):]
      else:
        y = autocorr_signal[str(i)][(j-50000):]
      break
    j += 1
  n = len(y)
  p = n - 1 
  while p > 0:
    d = np.abs(y[p,1] - y[(p-1),1])
    if d > 5:
      autocorr_signal[str(i)] = y[:(p+5000)]
      break
    p -= 1


In [0]:
# emphasize or high pass filtering step
emphasized_signal = {}
pre_emphasis = 0.975
for i in range(1,K):
  emphasized_signal[str(i)] = np.append(autocorr_signal[str(i)][0], autocorr_signal[str(i)][1:] - pre_emphasis * autocorr_signal[str(i)][:-1])


In [0]:
# resample 
signal = {}
from scipy.signal import resample 
for i in range(1,K):
  signal[str(i)]=resample(emphasized_signal[str(i)], 40000 , t=None, axis=0, window=None)


In [0]:
# framing
frame_size = 0.025
frame_stride = 0.01
frame_length = {}
frame_step = {}
signal_length = {}
frame_length = {}
num_frames = {}
pad_signal_length = {}
z = {}
pad_signal = {}
indices = {}
frames = {}

for i in range(1, K):
  frame_length[str(i)], frame_step[str(i)] = frame_size * sample_rate[str(i)], frame_stride * sample_rate[str(i)]  # Convert from seconds to samples
  signal_length[str(i)] = len(signal[str(i)])
  frame_length[str(i)] = int(round(frame_length[str(i)]))
  frame_step[str(i)] = int(round(frame_step[str(i)]))
  num_frames[str(i)] = int(np.ceil(float(np.abs(signal_length[str(i)] - frame_length[str(i)])) / frame_step[str(i)]))  # MaKe sure that we have at least [str(i)] frame

  pad_signal_length[str(i)] = num_frames[str(i)] * frame_step[str(i)] + frame_length[str(i)]
  z[str(i)] = np.zeros((pad_signal_length[str(i)] - signal_length[str(i)]))
  pad_signal[str(i)] = np.append(signal[str(i)], z[str(i)]) # Pad Signal to maKe sure that all frames have equal number of samples without truncating any samples from the original signal

  indices[str(i)] = np.tile(np.arange(0, frame_length[str(i)]), (num_frames[str(i)], 1)) + np.tile(np.arange(0, num_frames[str(i)] * frame_step[str(i)], frame_step[str(i)]), (frame_length[str(i)], 1)).T
  frames[str(i)] = pad_signal[str(i)][indices[str(i)].astype(np.int32, copy=False)]
  

In [0]:
# windowing with hamming window
for i in range(1,K):
  frames[str(i)] *= np.hamming(frame_length[str(i)])


In [0]:
# fft and power spectrum
NFFT = 512
mag_frames = {}
pow_frames = {}

for i in range(1,K):
  mag_frames[str(i)] = np.absolute(np.fft.rfft(frames[str(i)], NFFT))  # Magnitude of the FFT
  pow_frames[str(i)] = ((1.0 / NFFT) * ((mag_frames[str(i)]) ** 2))  # Power Spectrum
  


In [0]:
# filter bank forming
nfilt = 40
high_freq_mel = {}
mel_points = {}
hz_points = {}
biin = {}
fbank = {}
f_m_minus = {}
f_m = {}
f_m_plus = {}
filter_banks = {}

low_freq_mel = 0
for i in range(1,K):
  high_freq_mel[str(i)] = (2595 * np.log10(1 + (sample_rate[str(i)] / 2) / 700))  # Convert Hz to Mel
  mel_points[str(i)] = np.linspace(low_freq_mel, high_freq_mel[str(i)], nfilt + 2)  # Equally spaced in Mel scale
  hz_points[str(i)] = (700 * (10**(mel_points[str(i)] / 2595) - 1))  # Convert Mel to Hz
  biin[str(i)] = np.floor((NFFT + 1) * hz_points[str(i)] / sample_rate[str(i)])

  fbank[str(i)] = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
  for m in range(1, nfilt + 1):
      f_m_minus[str(i)] = int(biin[str(i)][m - 1])   # left
      f_m[str(i)] = int(biin[str(i)][m])             # center
      f_m_plus[str(i)] = int(biin[str(i)][m + 1])    # right

      for k in range(f_m_minus[str(i)], f_m[str(i)]):
          fbank[str(i)][m - 1, k] = (k - biin[str(i)][m - 1]) / (biin[str(i)][m] - biin[str(i)][m - 1])
      for k in range(f_m[str(i)], f_m_plus[str(i)]):
          fbank[str(i)][m - 1, k] = (biin[str(i)][m + 1] - k) / (biin[str(i)][m + 1] - biin[str(i)][m])
  filter_banks[str(i)] = np.dot(pow_frames[str(i)], fbank[str(i)].T)
  filter_banks[str(i)] = np.where(filter_banks[str(i)] == 0, np.finfo(float).eps, filter_banks[str(i)])  # Numerical Stability
  filter_banks[str(i)] = 20 * np.log10(filter_banks[str(i)])  # dB


In [0]:
# dct
num_ceps = 12
mfcc = {}
for i in range(1,K):
  mfcc[str(i)] = dct(filter_banks[str(i)], type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13

In [0]:
# mfcc
cep_lifter = 22
nframes = {}
ncoeff = {}
n = {}
lift = {}

for i in range(1,K):
  (nframes[str(i)], ncoeff[str(i)]) = mfcc[str(i)].shape
  n[str(i)] = np.arange(ncoeff[str(i)])
  lift[str(i)] = 1 + (cep_lifter / 2) * np.sin(np.pi * n[str(i)] / cep_lifter)
  mfcc[str(i)] *= lift[str(i)]  #*


In [0]:
# mean normalization
for i in range(1,K):
  mfcc[str(i)] -= (np.mean(mfcc[str(i)], axis=0) + 1e-8)

In [0]:
# dataset construction
train = {}
test = {}
p = 1
q = 1
for i in range(1,K):
  if (i%3) == 0:
    test[str(p)] = np.reshape(mfcc[str(i)],(1,mfcc[str(i)].shape[0]*mfcc[str(i)].shape[1]))
    p += 1
  else:
    train[str(q)] = np.reshape(mfcc[str(i)],(1,mfcc[str(i)].shape[0]*mfcc[str(i)].shape[1]))
    q += 1
  

In [0]:
feature = mfcc[str(2)].shape[0]*mfcc[str(2)].shape[1]

In [0]:
# concatenation
X = np.empty((1,feature))
for i in range(1, q):
  X = np.append(X,train[str(i)],0)
X = np.delete(X,[0],0)

In [0]:
# concatenation
T = np.empty((1,feature))
for i in range(1,p):
  T = np.append(T,test[str(i)],0)
T = np.delete(T,[0],0)

In [0]:
# truth value formation
Y_train = []
for i in range(1,q):
  if i <= 10:
    Y_train.append("1")
  elif (i>10) and (i<=20):
    Y_train.append("2")
  elif (i>20) and (i<=30):
    Y_train.append("3")
  elif (i>30) and (i<=40):
    Y_train.append("4")
  else:
    Y_train.append("5")
  

In [0]:
# truth value formation
Y_test = []
for i in range(1,p):
  if i<= 5:
    Y_test.append("1")
  elif (i>5) and (i<=10):
    Y_test.append("2")
  elif (i>10) and (i<=15):
    Y_test.append("3")
  elif (i>15) and (i<=20):
    Y_test.append("4")
  else:
    Y_test.append("5")

In [0]:
from sklearn.neighbors import KNeighborsClassifier

In [0]:
# KNN classifier
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X, Y_train)

In [0]:
# training prediction
x = model.predict(X)

In [0]:
# train accuracy
c = 0
for i in range(q-1):
  if (x[i] == Y_train[i]):
    c += 1
train_accuracy = (c/(q-1))*100
train_accuracy

In [0]:
# test preidiction
t = model.predict(T)

In [0]:
# test accuracy
c = 0
for i in range(p-1):
  if (t[i] == Y_test[i]):
    c += 1
    print(Y_test[i])
test_accuracy = (c/(p-1))*100
test_accuracy

In [0]:
sr,s = sciwave.read("5Recording (15).wav")

In [0]:
y = model.predict(train["40"])
y

In [0]:
train["40"].shape

In [0]:
import pickle

In [0]:
filename = 'modelwithKNN.sav'
pickle.dump(model, open(filename, 'wb'))

In [0]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(train["40"])
result